# Relation Between Mean-CVaR and Mean-Variance Optimization
This notebook illustrates the relationship between mean-CVaR and mean-variance optimization.

Like Proposition 1 in https://doi.org/10.21314/JOR.2000.038, we believe it is a desirable feature if mean-CVaR and mean-variance optimization give the same results when returns follow a multivariate normal distribution and only deviate significantly if the left tails of the P&L simulations deviate significantly from a normal distribution. However, this is not the case for mean-CVaR optimization with non-demeaned CVaR. To illustrate this, we simulate 1,000,000 (!) scenarios from a multivariate normal distribution and show that demeaned mean-CVaR and mean-variance indeed converge to practically the same results. However, the non-demeaned CVaR converges to a significantly different solution, which clearly prefers instruments with higher expected returns. Hence, the difference in optimization results can be caused simply by the expected return and not the difference in the left tail properties of the P&L simulation if one uses non-demeaned CVaR in mean-CVaR optimization. Unfortunately, demeaning CVaR for mean-CVaR optimization does not seem to be a current industry standard.

In [1]:
import numpy as np
import pandas as pd
import fortitudo.tech as ft
from time import time

In [2]:
# Load parameters and simulate multivariate normal P&L
instrument_names, means, covariance_matrix = ft.load_parameters()
S = 1000000
R = np.random.default_rng(1).multivariate_normal(means, covariance_matrix, S)

means_sim = np.mean(R, axis=0)
print(f'Max absolute difference for means is {np.max(np.abs(means_sim - means))}.')
covariances_sim = np.cov(R, rowvar=False)
print(f'Max absolute difference for covariances is {np.max(np.abs(covariances_sim - covariance_matrix))}.')

Max absolute difference for means is 0.0004694984202286423.
Max absolute difference for covariances is 6.967627212783184e-05.


In [3]:
# Specify long-only constraints
I = len(instrument_names)
A = np.ones((1, I))
b = np.array([1.])
G = -np.eye(I)
h = np.zeros(I)

# Specify optimization objects and compute efficient frontiers
opt_cvar = ft.MeanCVaR(R, A, b, G, h)
opt_cvar_mean = ft.MeanCVaR(R, A, b, G, h, options={'demean': False})
opt_var = ft.MeanVariance(means_sim, covariances_sim, A, b, G, h)
num_portfolios = 9
start_time = time()
frontier_cvar = opt_cvar.efficient_frontier(num_portfolios)
print(f'Demeaned CVaR efficient frontier with {S} scenarios, {I} instruments, and {num_portfolios} portfolios'
    + f' computed in {np.round(time() - start_time, 2)} seconds.')
start_time = time()
frontier_cvar_mean = opt_cvar_mean.efficient_frontier(num_portfolios)
print(f'Non-demeaned CVaR efficient frontier with {S} scenarios, {I} instruments, and {num_portfolios} portfolios' 
    + f' computed in {np.round(time() - start_time, 2)} seconds.')
frontier_var = opt_var.efficient_frontier(num_portfolios)

Demeaned CVaR efficient frontier with 1000000 scenarios, 10 instruments, and 9 portfolios computed in 18.12 seconds.
Non-demeaned CVaR efficient frontier with 1000000 scenarios, 10 instruments, and 9 portfolios computed in 17.64 seconds.


In [4]:
# Print efficient frontiers and compare
display(pd.DataFrame(np.round(100 * frontier_cvar, 2), index=instrument_names))
display(pd.DataFrame(np.round(100 * frontier_var, 2), index=instrument_names))
display(pd.DataFrame(np.round(100 * frontier_cvar_mean, 2), index=instrument_names))

,0,1,2,3,4,5,6,7,8
Gov & MBS,70.91,48.28,17.89,0.00,0.00,0.00,-0.00,-0.00,0.0
Corp IG,3.22,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.0
Corp HY,3.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Debt,0.00,0.18,8.12,1.77,-0.00,0.00,0.00,0.00,0.0
DM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Private Equity,0.00,-0.00,0.00,3.96,20.48,37.43,57.14,78.57,100.0
Infrastructure,1.82,9.46,16.68,28.27,40.40,52.54,42.86,21.43,-0.0
Real Estate,9.63,13.54,17.47,16.72,10.76,4.71,0.00,0.00,0.0
Hedge Funds,10.98,28.54,39.84,49.28,28.36,5.32,0.00,0.00,0.0


,0,1,2,3,4,5,6,7,8
Gov & MBS,70.92,48.19,17.86,0.00,0.00,0.00,0.00,0.00,0.0
Corp IG,3.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Corp HY,3.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Debt,0.00,0.35,8.25,2.14,0.00,0.00,0.00,0.00,0.0
DM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0
Private Equity,0.00,0.00,0.00,4.03,20.46,37.40,57.15,78.58,100.0
Infrastructure,1.76,9.48,16.84,28.44,40.62,52.75,42.85,21.42,0.0
Real Estate,9.61,13.32,17.08,16.26,10.49,4.36,0.00,0.00,0.0
Hedge Funds,11.15,28.65,39.97,49.13,28.43,5.49,0.00,0.00,0.0


,0,1,2,3,4,5,6,7,8
Gov & MBS,62.14,35.11,6.53,-0.00,0.00,-0.00,-0.0,-0.0,0.0
Corp IG,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,-0.0
Corp HY,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
EM Debt,0.00,3.63,11.22,0.00,0.00,0.00,-0.0,0.0,0.0
DM Equity,-0.00,0.00,0.00,0.00,0.00,-0.00,0.0,0.0,0.0
EM Equity,0.00,-0.00,-0.00,0.00,0.00,0.00,0.0,0.0,0.0
Private Equity,0.00,-0.00,0.00,8.79,24.68,40.57,59.8,79.9,100.0
Infrastructure,5.46,12.62,19.37,32.06,43.39,54.89,40.2,20.1,0.0
Real Estate,11.51,15.11,18.91,15.01,9.21,3.50,0.0,0.0,0.0
Hedge Funds,20.89,33.53,43.97,44.15,22.72,1.04,0.0,0.0,0.0


# License

In [5]:
# fortitudo.tech - Novel Investment Technologies.
# Copyright (C) 2021 Fortitudo Technologies ApS.

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.